In [17]:
from utilities import Utils
from feature_extraction import Feature
from forest_regression import run_forest
from forest_regression import RegressionForest
from sklearn.externals import joblib
from amrafile import amrafile as af
import json
import numpy as np

''' Init empty lists for storing reduced prototypes, reduced target and ncc'''
reduced_prototypes, reduced_targets, reduced_masks, ncc = [], [], [], []
reduced_mask = np.zeros((self._target_size), dtype = bool)

''' Calculate ncc between reduced target and reduced prototype space. 
    Reduced spaces will be of size 2*reduced_size+1 to get an odd kernel and a well defined center point''' 

reduced_size = self._reduced_size

for ind, poi in enumerate(prototype_pois):

    z_lower = poi[0]-reduced_size[0]
    z_upper = poi[0]+reduced_size[0]+1
    y_lower = poi[1]-reduced_size[1]
    y_upper = poi[1]+reduced_size[1]+1
    x_lower = poi[2]-reduced_size[2]
    x_upper = poi[2]+reduced_size[2]+1

    prototype = prototype_data[ind]

    ''' Extract reduced space from prototype and target'''
    reduced_prototype = prototype[z_lower:z_upper, y_lower:y_upper, x_lower:x_upper]
    reduced_target = self._water_data[z_lower:z_upper, y_lower:y_upper, x_lower:x_upper]

    ''' Create binary mask of the reduced space'''
    reduced_mask_copy = reduced_mask.copy()
    reduced_mask_copy[z_lower:z_upper, y_lower:y_upper, x_lower:x_upper] = True

    ''' Append reduced numpy arrays to lists'''
    #reduced_prototypes.append(reduced_prototype)
    #reduced_targets.append(reduced_target)
    reduced_masks.append(reduced_mask_copy)

            ''' Calculate ncc and store in lists'''
            ncc.append(normalized_cross_correlation(reduced_prototype, reduced_target))

        ''' Find index of poi which corresponds to highest ncc'''
        poi_index = ncc.index(max(ncc))    

        ''' Extract reduced data corresponding to highest ncc'''
        reduced_mask = reduced_masks[poi_index]

        reduced_water = np.reshape(self._water_data[reduced_mask], (2*self._reduced_size+1))
        reduced_fat = np.reshape(self._fat_data[reduced_mask], (2*self._reduced_size+1))

        return reduced_water, reduced_fat, reduced_mask, poi_index

In [26]:
test_target = '0019D'
target_path = '/moria/data/DB/'+directory+test_target+'/wholebody_normalized_water_1_'+test_target+'.amra'

In [27]:
''' Create utils class object '''
utils = Utils(target_path, search_size, extension, poi)

''' Init POI as just the ground truth + noise to reduce training time'''
reduced_data, reduced_mask = utils.simple_search_reduction()

''' Extract testing ground truth '''
ground_truth = utils.extract_ground_truth(reduced_mask)

''' Create feature object '''
feature = Feature(nbr_of_filters, patch_size)

''' Extract testing features '''
test_features = feature.feature_extraction(reduced_data, filter_bank, filter_parameters)

''' Run test data through forest '''
regression = run_forest(estimators, test_features)

poi_pos = utils.extract_poi_position(regression, reduced_mask)

print(poi_pos)


Ground truth:
[152 103 148]
Feature extraction
3.7029006481170654
Run forest
2.175846576690674
5.20849553146
25066
[153 104 149]
